In [1]:
# from google colab example notebooks
# from google.colab import drive, userdata
# drive.mount('/gdrive')

import os
from roboflow import Roboflow
from pathlib import Path
from ultralytics import YOLO, RTDETR
from pprint import pprint
from sahi import AutoDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction, predict
import wandb
from wandb.integration.ultralytics import add_wandb_callback
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

PROJECT_DIR = Path('/home/camille/code/capstone')
os.chdir(PROJECT_DIR)
# %cd /gdrive/MyDrive/capstone
# %cd /gdrive/MyDrive/capstone
# %ls

WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")



In [2]:
wandb.login(key = os.getenv('WANDB_KEY'))

11/14/2023 01:02:25 - ERROR - wandb.jupyter -   Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cseaberry (seaberry). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/camille/.netrc


True

In [3]:
from ultralytics import settings as ul_settings
# ul_settings.update({ 'datasets_dir': '/gdrive/MyDrive/capstone/datasets' })
ul_settings

{'settings_version': '0.0.4',
 'datasets_dir': '/home/camille/code/capstone/data',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '82876d9ba7845817dc4101ed78893f7c8b224fb5021052403613ed862296ada3',
 'sync': True,
 'api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [4]:
def download_rf_imgs(proj, v, data_dir, format, overwrite = False):
  dataset = proj.version(v).download(model_format = format, location = data_dir, overwrite = overwrite)
  return dataset

rf = Roboflow(api_key = os.getenv('ROBOFLOW_KEY'))
proj = rf.workspace('seaberry').project('cap-detect')
dataset_full = download_rf_imgs(proj, v = 2, 
                                data_dir = str(PROJECT_DIR / 'data/cams_full'), 
                                format = 'yolov8', 
                                overwrite = False) # main full images
dataset_tile = download_rf_imgs(proj, v = 3, 
                                data_dir = str(PROJECT_DIR / 'data/cams_tile'), 
                                format = 'yolov8', 
                                overwrite = False) # tiled images

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.186, to fix: `pip install ultralytics==8.0.134`
Dependency ultralytics==8.0.134 is required but found version=8.0.186, to fix: `pip install ultralytics==8.0.134`


In [5]:
import yaml
from pathlib import Path

def fix_data_yaml(dir):
    '''
    Paths to image directories have been inconsistent. This gives them consistent paths
    such as train/images, rather than longer relative ones.
    '''
    dir = Path(dir)
    with open(dir / 'data.yaml', 'r') as f:
        data = yaml.safe_load(f)
    for split in ['train', 'val', 'test']:
        long_dir = Path(data[split])
        data[split] = long_dir.parent.name + '/' + long_dir.name
    with open(dir / 'data.yaml', 'w') as f:
        yaml.dump(data, f)

for dset in [dataset_full, dataset_tile]:
    fix_data_yaml(dset.location)

In [6]:
%cat data/cams_full/data.yaml
!echo ""
%cat data/cams_tile/data.yaml

names:
- surveillance
nc: 1
roboflow:
  license: CC BY 4.0
  project: cap-detect
  url: https://universe.roboflow.com/seaberry/cap-detect/dataset/2
  version: 2
  workspace: seaberry
test: test/images
train: train/images
val: valid/images

names:
- surveillance
nc: 1
roboflow:
  license: CC BY 4.0
  project: cap-detect
  url: https://universe.roboflow.com/seaberry/cap-detect/dataset/3
  version: 3
  workspace: seaberry
test: test/images
train: train/images
val: valid/images


In [7]:
%ls -1 data/cams_tile/train/images | wc -l

7794


In [8]:
params = {}
yolo = YOLO('yolov8s.pt')
detr = RTDETR('rtdetr-l.pt')
params['yolo_full'] = { 'dataset': dataset_full, 'model': yolo }
params['detr_full'] = { 'dataset': dataset_full, 'model': detr }
# params['yolo_tile'] = { 'dataset': dataset_tile, 'model': yolo }
# params['detr_tile'] = { 'dataset': dataset_tile, 'model': detr }

def model_with_wb(
    id,
    model,
    dataset,
    project = 'capstone',
    imgsz = 640,
    patience = 10,
    epochs = 5,
    batch = 16,
    exist_ok = True,
    **kwargs
):
    data_path = f'{dataset.location}/data.yaml'
    with wandb.init(project = project, name = id) as run:
        model.train(data = data_path,
                    imgsz = imgsz,
                    patience = patience,
                    epochs = epochs,
                    batch = batch,
                    exist_ok = exist_ok,
                    single_cls = True,
                    name = f'{id}_train',
                    **kwargs)
        model.val()

In [9]:
# model_with_wb('yolo_full', yolo, dataset_full, epochs = 5, batch = 16, freeze = 20, save = False, amp = False)
# model_with_wb('yolo_tile', yolo, dataset_tile, epochs = 5)
for key, val in params.items():
    model_with_wb(key, val['model'], val['dataset'], epochs = 5, batch = 16, freeze = 20, save = True, amp = False)

New https://pypi.org/project/ultralytics/8.0.209 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.186 🚀 Python-3.11.5 torch-2.1.0 CUDA:0 (NVIDIA T550 Laptop GPU, 3906MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/camille/code/capstone/data/cams_full/data.yaml, epochs=5, patience=10, batch=-1, imgsz=640, save=False, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolo_full_train, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=20, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=N

In [ ]:
mod_yolo = YOLO('yolov8s.pt')
mod_detr = RTDETR('rtdetr-l.pt')
data_path = f'{dataset.location}/data.yaml'

with wandb.init(project = 'capstone', name = 'yolo_unsliced') as yolo_run:
    mod_yolo.train(data = data_path,
                   imgsz = 640,
                   patience = 10,
                   epochs = 5,
                   exist_ok = True,
                   name = 'yolo8s_train')


In [ ]:
with wandb.init(project = 'capstone', name = 'detr_unsliced') as detr_run:
    mod_detr.train(data = data_path,
                   imgsz = 640,
                   patience = 10,
                   epochs = 5,
                   exist_ok = True,
                   name = 'detr_train')

In [ ]:
wandb.finish()